# Convolution

### Pytorch visualization : visdom



### visdom 사용법 
- Text
- Image
- Images 
- Line plot 

여기선 visdom 사용법을 익히고  MNIST-CNN loss graph 까지 적용 하도록 한다.

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

In [3]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


만약 위 코드에서 에러를 발생한다면, 터미널에서

- python -m visdom.server

위 코드를 쳐주면 된다.

### Text

In [4]:
vis.text('hello world', env="main")

'window_3801e0d67dbf38'

### Image

In [5]:
a = torch.randn(3,200,200) # 앞의 3은 rgb
vis.image(a)

'window_3801e10b112b42'

In [6]:
vis.images(torch.Tensor(3,3,28,28))

'window_3801e1480a0fd4'

### example (using MNIST and CIFAR10)

In [7]:
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

100%|█████████████████████████████████████████████████████████████▉| 170418176/170498071 [00:13<00:00, 14720154.45it/s]

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


In [8]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_3801e1888cae4c'

170500096it [00:30, 14720154.45it/s]                                                                                   

In [9]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_3801e192ed7a64'

### 한번에 여러 MNIST 이미지 보기 

In [11]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32, 
                                          shuffle = False
                                         )

In [15]:
for num,value in enumerate(data_loader) :
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


### Line Plot
학습을 진행하면서 로스값을 그래프로 확인하는것이 가능하다

그전에 아래 명령어를 통해 지금까지 실행했던 visdom 이미지들을 다 닫도록 하겠습니다.

In [20]:
vis.close(env="main")

''

In [21]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [22]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data,X=X_data)

### Line Update

In [24]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6]) # 인덱스 값

vis.line(Y=Y_append, X=X_append,win=plt, update="append")

'window_3801e3c5798ef2'

### Multiple Line on single window 
단일 라인플롯 창에서 두개의 라인 그리기

In [27]:
num = torch.Tensor(list(range(0,10))) # 10개짜리 텐서 하나 생성
print(num.shape)
num = num.view(-1,1) # 
print(num.shape)

num = torch.cat((num,num),dim = 1)
print(num.shape)

plt = vis.line(Y=torch.randn(10,2), X = num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


### Line Info

In [28]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [29]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [30]:
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

### Make function for Update Line

자 그러면 우리가 실제로 이제 실제로 로스값에 대해 어떻게 업데이트가 되는지를 알아보고 싶을것이다.

In [31]:
def loss_tracker(loss_plot, loss_value, num): 
    #loss_plot : 업데이트할 플롯, loss_value : 로스값이 무엇인지, num : x값을 의미
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [32]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [33]:
vis.close(env="main")

''

위의 것을 응용하여 구할수 있다.

In [34]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [35]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [36]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)

cuda


In [38]:
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

In [40]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [41]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [42]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

In [43]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [44]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


In [45]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### make plot

In [46]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [47]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    # loss_tracker 가 loss_plt를 업데이트 할것입니다. 
    # avg_cost를 업데이트 할것이며, 
    # x_index는 epoch로 하겠습니다 .
    
print('Learning Finished!')

[Epoch:1] cost = 0.12048943340778351
[Epoch:2] cost = 0.04065356031060219
[Epoch:3] cost = 0.028350068256258965
[Epoch:4] cost = 0.022041650488972664
[Epoch:5] cost = 0.017645344138145447
[Epoch:6] cost = 0.014207673259079456
[Epoch:7] cost = 0.012411192990839481
[Epoch:8] cost = 0.01135083194822073
[Epoch:9] cost = 0.008560414426028728
[Epoch:10] cost = 0.009153187274932861
[Epoch:11] cost = 0.007977004162967205
[Epoch:12] cost = 0.007860653102397919
[Epoch:13] cost = 0.00791374035179615
[Epoch:14] cost = 0.007302328012883663
[Epoch:15] cost = 0.0069701168686151505
Learning Finished!


In [48]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

C:\Users\ashgh\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\ashgh\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9472000002861023
